In [43]:
# Importing data manipulation/visualization packages
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Import ML framework library
import torch as th
import torch.nn as nn
import torch.nn.init as init

# Importing gym packages
import gymnasium as gym
from gymnasium import spaces

# Importing IRL libraries
from stable_baselines3.ppo import MlpPolicy
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3 import PPO

# Importing imitation library
from imitation.algorithms.adversarial.airl import AIRL
from imitation.util import util
from imitation.data import rollout
from imitation.data.wrappers import RolloutInfoWrapper
from imitation.util.util import make_vec_env
from imitation.rewards.reward_nets import BasicShapedRewardNet
from imitation.rewards.reward_nets import RewardNet
from imitation.util.networks import RunningNorm
from imitation.util import networks, util

# Import miscellaneuous packages
import random
from scipy.stats import norm
from scipy.stats import bernoulli

In [44]:
# Setting the seed
SEED = 42
np.random.seed(SEED)
th.manual_seed(SEED)
if th.cuda.is_available():
    th.cuda.manual_seed_all(SEED)

In [45]:
# Arbitrary weights for the importance of the engagement level and section number
a, b = 1, 0.5

# Arbitary thresholds for what to what scores should be considered extremely interested, mildy interested, and not interested
theta_1 = 0.9
theta_2 = 0.4

# Define actions
ACTION_WAIT = 0
ACTION_READ_FREE = 1
ACTION_READ_PAY = 2

# Define sections
BEGINNING = 0
MIDDLE = 1
END = 2

# Gamma decay for hours waited (in days)
gamma = 0.1

# Number of chapters
NUM_CHAPTERS = 24

In [46]:
# Helper function to calculate how interested an agent is
def interest_score (engagement_level, section_number):
    """
    Calculates a metric to gauge engagement using state feature information.
    :param options: Engagement level and the section number (chapter number)
    :return: Calculated score
    """
    norm_section_number = (section_number - 1) / (NUM_CHAPTERS - 1)
    return a * engagement_level + b * norm_section_number

In [47]:
# Helper function to draw from Bernoulli (in my case, want to draw to choose between 2 values)
def probabilistic_choice(options):
    """
    Chooses an action probabilistically based on weights.
    :param options: List of (action, probability) tuples
    :return: Chosen action
    """
    actions, probabilities = zip(*options)
    return random.choices(actions, weights=probabilities, k=1)[0]

In [48]:
def find_section(chapter):
    """
    Outputs whether the chapter is in the beginning, middle, or end of the book.
    :param options: Chapter
    :return: BEGINNING, MIDDLE, or END
    """
    if chapter <= 1/3 * NUM_CHAPTERS:
        return BEGINNING
    elif 1/3 * NUM_CHAPTERS < chapter <= 2/3 * NUM_CHAPTERS:
        return MIDDLE
    else:
        return END


In [49]:
def true_reward (state, action):
	"""
	The true reward function that maps a (state, action) pair to a real number.
	:param options: A state feature list [section_number, engagement_level, time, price, wff, wff_hours_required, wff_hours_waited], and an action
	:return: A real number (the reward)
	"""	
	
	section_number, engagement_level, time, price, wff, wff_hours_required, wff_hours_waited = state
	section = find_section(section_number)
	score = interest_score(engagement_level, section_number)
	
	if section == BEGINNING:
		if score > theta_1:
			if price == 0:
				if action == ACTION_READ_FREE:
					return 9
				else:
					return -1
		elif theta_2 < score < theta_1:
			if price == 0:
				if action == ACTION_READ_FREE:
					return 7
				else:
					return -1
		else:
			if price == 0:
				if action == ACTION_READ_FREE:
					return 5
				else:
					return -1
	elif section == MIDDLE:
		if wff:
			return 6 * (1 - gamma) ** (wff_hours_waited / 24)
		else:
			if score > theta_1:
				if price == 1:
					if action == ACTION_READ_PAY:
						return 10
					else:
						return -1
			elif theta_2 < score < theta_1:
				if price == 0:
					if action == ACTION_READ_PAY:
						return 8
					else:
						return -1
			else:
				if price == 0:
					if action == ACTION_READ_PAY:
						return 6
					else:
						return -1
	else:
		if score > theta_1:
			if price == 1:
				if action == ACTION_READ_PAY:
					return 11
				else:
					return -1
		elif theta_2 < score < theta_1:
			if price == 1:
				if action == ACTION_READ_PAY:
					return 9
				else:
					return -1
		else:
			if price == 1:
				if action == ACTION_READ_PAY:
					return 7
				else:
					return -1
	return -5

In [50]:
class EBookEnv(gym.Env):
    def __init__(self):
        super(EBookEnv).__init__()

        # state =  [section_number, engagement_level, time, price, wff, wff_hours_required, wff_hours_waited]
        self.observation_space = spaces.Box(low=0, high=1, shape=(7,), dtype=np.float32)

        # action = [wait, read_without_payment, read_with_payment]
        self.action_space = spaces.Discrete(3)

        self.state = None
        self.times_bought = 0

    def reset(self, seed=None, options=None):
        # Resetting the state so that the reader begins at section 1, at a 0.8 engagement level, time 0, price 0, wff 0, and the wff hours stuff 0
        self.state = np.array([1, 0.8, 0, 0, 0, 0, 0], dtype=np.float32)
        self.times_bought = 0
        return self.state, {}

    def step(self, action):
        # Calculate the reward
        reward = true_reward(self.state, action)

        section_number, engagement_level, time, price, wff, wff_hours_required, wff_hours_waited = self.state
        section = find_section(section_number)

        if section == BEGINNING:
            if action == ACTION_READ_FREE:
                section_number += 1
                time = 0
            engagement_level = norm.cdf(np.random.normal(0, 1)) # Draw the engagement index from a N~(0, 1) and normalize to [0, 1]
            if find_section(section_number) == BEGINNING:
                price = 0
            else:
                price = 1
        elif section == MIDDLE:
            if wff == 0:
                if action == ACTION_READ_PAY:
                    section_number += 1
                    if find_section(section_number) == MIDDLE:
                        wff = np.random.choice([0, 1], p=[0.5, 0.5])
                        if wff:
                            wff_hours_required = np.random.randint(1, 4)
                            wff_hours_waited = 0
                    engagement_level = norm.cdf(np.random.normal(0, 1)) # Draw the engagement index from a N~(0, 1) and normalize to [0, 1]
                    time = 0
                price = 1
                time += 1
            else:
                if ACTION_READ_PAY:
                    section_number += 1
                    wff = 0
                    wff_hours_required = 0
                    wff_hours_waited = 0
                    time += 1
                    engagement_level = norm.cdf(np.random.normal(0, 1)) # Draw the engagement index from a N~(0, 1) and normalize to [0, 1]
                    price = 1
                else:
                    wff_hours_waited += 1
                    time += 1
                    if wff_hours_waited == wff_hours_required:
                        wff = 0
        else:
            if action == ACTION_READ_PAY:
                section_number += 1
                time = 0
            price = 1
            engagement_level = norm.cdf(np.random.normal(0, 1)) # Draw the engagement index from a N~(0, 1) and normalize to [0, 1]
            time += 1
        # Save the new state
        self.state = np.array([section_number, engagement_level, time, price, wff, wff_hours_required, wff_hours_waited], dtype=np.float32)

        # The terminating condition is when you reach the end of a book or if the time interval gets very large
        done = time >= 120 or section_number > NUM_CHAPTERS

        truncated = False
        info = {"obs": self.state, "rews": reward}
        return self.state, float(reward), done, truncated, info

    def render(self, mode='human'):
        pass


In [51]:
# Register the gym for compatability with OpenAI Gym
gym.register(id='EBookEnv-v1', entry_point=lambda: EBookEnv())
venv = util.make_vec_env("EBookEnv-v1", rng=np.random.default_rng(SEED), n_envs=1, post_wrappers=[lambda env, _: RolloutInfoWrapper(env)])

/Users/williamhuang/anaconda3/envs/tobin/lib/python3.9/site-packages/gymnasium/envs/registration.py:694: UserWarning: WARN: Overriding environment EBookEnv-v1 already in registry.
  logger.warn(f"Overriding environment {new_spec.id} already in registry.")


In [52]:
expert = PPO("MlpPolicy", venv, verbose=1)
expert.learn(total_timesteps=10000)

Using cpu device
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 68.1     |
|    ep_rew_mean     | 81.8     |
| time/              |          |
|    fps             | 4785     |
|    iterations      | 1        |
|    time_elapsed    | 0        |
|    total_timesteps | 2048     |
---------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 61.1        |
|    ep_rew_mean          | 88.7        |
| time/                   |             |
|    fps                  | 3536        |
|    iterations           | 2           |
|    time_elapsed         | 1           |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.013774436 |
|    clip_fraction        | 0.195       |
|    clip_range           | 0.2         |
|    entropy_loss         | -1.09       |
|    explained_variance   | -0.000417   |
|    learning

In [53]:
# Collect 500 trajectories of the expert behavior 
trajectories = rollout.rollout(
    expert,
    venv,
    rollout.make_sample_until(min_episodes=500),
    rng=np.random.default_rng(SEED),
)


In [54]:
"""
    Initialize a PPO agent to interact with the environment. 
    In AIRL, the learner is the generator. The model will try to imitate expert behavior by 
    having the discriminator distinguish between trajectories generated between the PPO agent
    and the true trajectories from the expert policy defined above.
"""
learner = PPO(
    env=venv,
    policy=MlpPolicy,
    batch_size=64,
    ent_coef=0.0,
    learning_rate=0.0005,
    gamma=0.95,
    clip_range=0.1,
    vf_coef=0.1,
    n_epochs=5,
    seed=SEED,
)

"""
    Initialize a reward network. 
"""
reward_net = BasicShapedRewardNet(
    observation_space=venv.observation_space,
    action_space=venv.action_space,
    normalize_input_layer=RunningNorm,
)

# Intialize parameters for AIRL model
airl_trainer = AIRL(
    demonstrations=trajectories,
    demo_batch_size=2048,
    gen_replay_buffer_capacity=512,
    n_disc_updates_per_round=16,
    venv=venv,
    gen_algo=learner,
    reward_net=reward_net,
    allow_variable_horizon=True
)

Running with `allow_variable_horizon` set to True. Some algorithms are biased towards shorter or longer episodes, which may significantly confound results. Additionally, even unbiased algorithms can exploit the information leak from the termination condition, producing spuriously high performance. See https://imitation.readthedocs.io/en/latest/getting-started/variable-horizon.html for more information.


In [55]:
venv.seed(SEED)

airl_trainer.train(20000)  # Train for 2_000_000 steps to match expert by learning a reward and a policy

round:   0%|          | 0/9 [00:00<?, ?it/s]

------------------------------------------
| raw/                        |          |
|    gen/rollout/ep_len_mean  | 67.6     |
|    gen/rollout/ep_rew_mean  | 80.5     |
|    gen/time/fps             | 2136     |
|    gen/time/iterations      | 1        |
|    gen/time/time_elapsed    | 0        |
|    gen/time/total_timesteps | 2048     |
------------------------------------------
--------------------------------------------------
| raw/                                |          |
|    disc/disc_acc                    | 0.5      |
|    disc/disc_acc_expert             | 1        |
|    disc/disc_acc_gen                | 0        |
|    disc/disc_entropy                | 0.568    |
|    disc/disc_loss                   | 0.823    |
|    disc/disc_proportion_expert_pred | 1        |
|    disc/disc_proportion_expert_true | 0.5      |
|    disc/global_step                 | 1        |
|    disc/n_expert                    | 2.05e+03 |
|    disc/n_generated                 | 2.05e+03 |
-

round:  11%|█         | 1/9 [00:01<00:13,  1.75s/it]

----------------------------------------------------
| raw/                               |             |
|    gen/rollout/ep_len_mean         | 68.1        |
|    gen/rollout/ep_rew_mean         | 72.9        |
|    gen/rollout/ep_rew_wrapped_mean | 55.8        |
|    gen/time/fps                    | 2338        |
|    gen/time/iterations             | 1           |
|    gen/time/time_elapsed           | 0           |
|    gen/time/total_timesteps        | 4096        |
|    gen/train/approx_kl             | 0.004087215 |
|    gen/train/clip_fraction         | 0.275       |
|    gen/train/clip_range            | 0.1         |
|    gen/train/entropy_loss          | -1.09       |
|    gen/train/explained_variance    | -0.013      |
|    gen/train/learning_rate         | 0.0005      |
|    gen/train/loss                  | 0.585       |
|    gen/train/n_updates             | 5           |
|    gen/train/policy_gradient_loss  | -0.00756    |
|    gen/train/value_loss            | 21.5   

round:  22%|██▏       | 2/9 [00:03<00:10,  1.53s/it]

...

round:  78%|███████▊  | 7/9 [00:10<00:02,  1.44s/it]

----------------------------------------------------
| raw/                               |             |
|    gen/rollout/ep_len_mean         | 54.1        |
|    gen/rollout/ep_rew_mean         | 100         |
|    gen/rollout/ep_rew_wrapped_mean | -51.3       |
|    gen/time/fps                    | 2140        |
|    gen/time/iterations             | 1           |
|    gen/time/time_elapsed           | 0           |
|    gen/time/total_timesteps        | 16384       |
|    gen/train/approx_kl             | 0.004776245 |
|    gen/train/clip_fraction         | 0.369       |
|    gen/train/clip_range            | 0.1         |
|    gen/train/entropy_loss          | -1          |
|    gen/train/explained_variance    | 0.535       |
|    gen/train/learning_rate         | 0.0005      |
|    gen/train/loss                  | 0.549       |
|    gen/train/n_updates             | 35          |
|    gen/train/policy_gradient_loss  | -0.0218     |
|    gen/train/value_loss            | 6.97   

round:  89%|████████▉ | 8/9 [00:11<00:01,  1.44s/it]

-----------------------------------------------------
| raw/                               |              |
|    gen/rollout/ep_len_mean         | 52.6         |
|    gen/rollout/ep_rew_mean         | 108          |
|    gen/rollout/ep_rew_wrapped_mean | -50.5        |
|    gen/time/fps                    | 1773         |
|    gen/time/iterations             | 1            |
|    gen/time/time_elapsed           | 1            |
|    gen/time/total_timesteps        | 18432        |
|    gen/train/approx_kl             | 0.0036382335 |
|    gen/train/clip_fraction         | 0.282        |
|    gen/train/clip_range            | 0.1          |
|    gen/train/entropy_loss          | -0.969       |
|    gen/train/explained_variance    | 0.543        |
|    gen/train/learning_rate         | 0.0005       |
|    gen/train/loss                  | 0.941        |
|    gen/train/n_updates             | 40           |
|    gen/train/policy_gradient_loss  | -0.0236      |
|    gen/train/value_loss   

round: 100%|██████████| 9/9 [00:13<00:00,  1.55s/it]


In [56]:
# Helper function to calculate the behavorial cloning loss
def behavioral_cloning_loss(expert_trajs, learner_policy, device='cpu'):
    """
    Calculates the behavioral cloning loss (a measurement of policy similarity) by using
    expert trajectories and the learned policy from AIRL.
    :param options: A list of trajectories, and a learned policy
    :return: Behavorial cloning loss
    """
    total_loss = 0
    num_samples = 0

    for traj in expert_trajs:
        states = traj.obs  # Observations (states) from the trajectory
        expert_actions = traj.acts  # Expert's actions
        
        # Convert states to PyTorch Tensors
        states_tensor = th.tensor(states, dtype=th.float32, device=device)
        
        # Get learner's actions from the policy
        with th.no_grad():
            learner_actions = learner_policy(states_tensor)[0]

        learner_actions_np = learner_actions.cpu().numpy()
        
    #     # Calculate the loss (number of mismatched actions)
        loss = np.sum(learner_actions_np != expert_actions)
        total_loss += loss
        num_samples += len(states)
    return total_loss / num_samples  # Average loss

bc_loss = behavioral_cloning_loss(trajectories, learner.policy)
print(f"Behavioral Cloning Loss: {bc_loss}")

Behavioral Cloning Loss: 0.028768699654775604


/var/folders/wp/q7yyy4ls2m3bvyscgh359t0r0000gn/T/ipykernel_13369/3342791352.py:26: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  loss = np.sum(learner_actions_np != expert_actions)
